In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

files = os.listdir(path='data/')

In [2]:
devices = ["Respeck", "Thingy"]
student_numbers = ["s1923862", "s1915704", "s1971889"]
eating_static_activities = ["Sitting", "Standing"]
static_activities = ["Lying down right", "Lying down on left", "Lying down on stomach", "Lying down back"]
dynamic_activities = ["Shuffle walking", "Normal walking", "Running", "Ascending stairs", "Descending stairs", "Miscellaneous movements"]
sub_activities = ["Normal", "Talking", "Singing", "Coughing", "Laughing", "Hyperventilating"]
eating_sub_activity = "Eating"

In [3]:
def get_new_rows(student_number, activity, sub_activity, is_static):
    file = f"{devices[0]}_{student_number}_{activity}_{sub_activity}.csv"
    filename = "data/"+ file
    df = pd.read_csv(filename, index_col='Unnamed: 0')
    accel_data = df[['accel_x', 'accel_y', 'accel_x']]
    x = list(accel_data.apply(np.linalg.norm, axis=1))
    new_rows = []
    for i in range(0, 760-50, 10):
        new_rows.append([activity, sub_activity, is_static, student_number]+x[i:i+50])
    return new_rows

dataset = pd.DataFrame(columns=['activity', 'sub_activity', 'is_static', 'student_number']+list(range(0, 50)))

for student_number in student_numbers:
    for activity in static_activities:
        for sub_activity in sub_activities:
            new_rows = get_new_rows(student_number, activity, sub_activity, 1)
            for new_row in new_rows:
                dataset.loc[len(dataset)] = new_row

    for activity in eating_static_activities:
        for sub_activity in sub_activities:
            new_rows = get_new_rows(student_number, activity, sub_activity, 1)
            for new_row in new_rows:
                dataset.loc[len(dataset)] = new_row
        new_rows = get_new_rows(student_number, activity, eating_sub_activity, 1)
        for new_row in new_rows:
            dataset.loc[len(dataset)] = new_row

    for activity in dynamic_activities:
        new_rows = get_new_rows(student_number, activity, sub_activities[0], 0)
        for new_row in new_rows:
            dataset.loc[len(dataset)] = new_row

In [4]:
dataset

,activity,sub_activity,is_static,student_number,0,1,2,3,4,5,...,40,41,42,43,44,45,46,47,48,49
0,Lying down right,Normal,1,s1923862,1.241628,1.240868,1.234217,1.245862,1.235141,1.237945,...,1.257212,1.245895,1.251277,1.248757,1.253103,1.255849,1.258513,1.256710,1.255101,1.250688
1,Lying down right,Normal,1,s1923862,1.242848,1.235273,1.234713,1.235639,1.240740,1.238234,...,1.256491,1.256012,1.257030,1.261137,1.258575,1.259534,1.255796,1.250319,1.255953,1.249203
2,Lying down right,Normal,1,s1923862,1.242561,1.235633,1.240620,1.238879,1.238650,1.233895,...,1.251468,1.251028,1.247657,1.254970,1.257513,1.244499,1.239584,1.240750,1.240353,1.239348
3,Lying down right,Normal,1,s1923862,1.252126,1.253751,1.257316,1.250660,1.249537,1.249518,...,1.240058,1.239106,1.233039,1.233873,1.243424,1.233987,1.242454,1.239846,1.235889,1.229554
4,Lying down right,Normal,1,s1923862,1.257212,1.245895,1.251277,1.248757,1.253103,1.255849,...,1.236154,1.236571,1.224718,1.231839,1.233077,1.236794,1.232433,1.233229,1.230206,1.231756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9367,Miscellaneous movements,Normal,0,s1971889,0.817013,0.706036,0.971095,1.376889,1.414787,1.221207,...,1.047781,1.091985,1.126488,1.305027,1.185623,1.018424,1.129385,1.124105,1.006815,1.000440
9368,Miscellaneous movements,Normal,0,s1971889,1.248636,1.413960,1.123355,0.911059,0.706130,0.653855,...,0.897684,0.784718,0.772830,2.072183,1.690989,1.088081,0.902618,0.638701,0.776363,1.110394
9369,Miscellaneous movements,Normal,0,s1971889,0.691438,0.738462,1.063657,1.327633,1.466869,1.138023,...,1.245876,1.550012,1.195462,0.785179,1.050991,1.334619,1.260226,1.193464,0.893621,0.742346
9370,Miscellaneous movements,Normal,0,s1971889,1.015512,0.973744,0.963490,0.906513,1.035292,1.387279,...,0.833292,0.910756,1.624435,1.663567,1.226395,0.915870,0.635461,0.654234,0.994734,1.266988


### Model 1 - Stationary vs Physical activities

In [6]:
Xs = dataset[list(range(50))].values
ys = dataset['is_static']

X_trn, X_tst, y_trn, y_tst = train_test_split(Xs, ys, test_size=0.3, random_state=1, shuffle=True)

stationary_vs_physical_model = LogisticRegression(random_state=0, max_iter=1000).fit(X_trn, y_trn)

stationary_vs_physical_model.score(X_tst, y_tst)

0.8669985775248933

### Model 2 - Classify stationary activities

### Model 3 - Is respiratory sub-activity

### Model 4 - Classify Physical activities